In [2]:
class NumArray:

    def __init__(self, nums: List[int]):
        self.nums = nums
        self.n = len(nums)
        self.tree = [0] * (self.n * 4)
        self.buildTree(0, self.n - 1, 0)
    
    def buildTree(self, start, end, node_idx):
        # 构建线段树
        # node_idx 表示树的第几号节点
        if start == end:
            self.tree[node_idx] = self.nums[start]
        else:
            mid = (start + end) // 2
            left_node = 2 * node_idx + 1 # 树左节点的编号
            right_node = 2 * node_idx + 2
            self.buildTree(start, mid, left_node)
            self.buildTree(mid+1, end, right_node)
            self.tree[node_idx] = self.tree[left_node] + self.tree[right_node]
        
    def update_tree(self, start, end, node, idx, val):
        if start == end:
            self.tree[node] = val
        else:
            mid = (start + end) // 2
            left_node = 2 * node + 1 # 树左节点的编号
            right_node = 2 * node + 2
            if start <= idx <= mid:
                self.update_tree(start, mid, left_node, idx, val)
            else:
                self.update_tree(mid+1, end, right_node, idx, val)
            self.tree[node] = self.tree[left_node] + self.tree[right_node]
    
    def update(self, index: int, val: int) -> None:
        # 树的根节点的编号为 0
        self.update_tree(0, self.n-1, 0, index, val)
        
    def sum_range(self, start, end, node, left, right):
        if left > end or right < start:
            return 0
        elif start == end or (start >= left and end <= right):
            return self.tree[node]
        else:
            mid = (start + end) // 2
            left_node = 2 * node + 1 # 树左节点的编号
            right_node = 2 * node + 2
            sum_left = self.sum_range(start, mid, left_node, left, right)
            sum_right = self.sum_range(mid+1, end, right_node, left, right)
            return sum_left + sum_right
            
    def sumRange(self, left: int, right: int) -> int:
        return self.sum_range(0, self.n-1, 0, left, right)


# Your NumArray object will be instantiated and called as such:
# obj = NumArray(nums)
# obj.update(index, val)
# param_2 = obj.sumRange(left, right)

In [ ]:
class NumArray:
    def __init__(self, nums: List[int]):
        self.nums = nums
        self.h = [0] * (1+len(nums))
        for i in range(len(nums)):
            self.inc(i, nums[i])

    def update(self, i: int, val: int) -> None:
        self.inc(i, val - self.nums[i])
        self.nums[i] = val

    def inc(self, i: int, val: int) -> None:
        i += 1
        while i <= len(self.nums):
            self.h[i] += val
            i += i & (-i)

    def sumRange(self, i: int, j: int) -> int:
        return self.sumToIndex(j) - self.sumToIndex(i - 1)

    def sumToIndex(self, i: int) -> int:
        res = 0
        i += 1
        while i > 0:
            res += self.h[i]
            i -= i & (-i)
        return res

In [ ]:
class SegmentTree:
    def __init__(self, data, merge): 
        self.data = data
        self.n = len(data)
        #  申请4倍data长度的空间来存线段树节点
        self.tree = [None] * (4 * self.n) # 索引i的左孩子索引为2i+1，右孩子为2i+2
        self._merge = merge
        if self.n:
            self._build(0, 0, self.n-1)

    def query(self, ql, qr):
        return self._query(0, 0, self.n-1, ql, qr)

    def update(self, index, value):
        # 将data数组index位置的值更新为value,然后递归更新线段树中被影响的各节点的值
        self.data[index] = value
        self._update(0, 0, self.n-1, index)

    def _build(self, tree_index, l, r):
        if l == r:
            self.tree[tree_index] = self.data[l]
            return
        mid = (l+r) // 2 # 区间中点,对应左孩子区间结束,右孩子区间开头
        left, right = 2 * tree_index + 1, 2 * tree_index + 2 # tree_index的左右子树索引
        self._build(left, l, mid)
        self._build(right, mid+1, r)
        self.tree[tree_index] = self._merge(self.tree[left], self.tree[right])

    def _query(self, tree_index, l, r, ql, qr):
        if l == ql and r == qr:
            return self.tree[tree_index]

        mid = (l+r) // 2 # 区间中点,对应左孩子区间结束,右孩子区间开头
        left, right = tree_index * 2 + 1, tree_index * 2 + 2
        if qr <= mid:
            # 查询区间全在左子树
            return self._query(left, l, mid, ql, qr)
        elif ql > mid:
            # 查询区间全在右子树
            return self._query(right, mid+1, r, ql, qr)

        # 查询区间一部分在左子树一部分在右子树
        return self._merge(self._query(left, l, mid, ql, mid), 
                          self._query(right, mid+1, r, mid+1, qr))

    def _update(self, tree_index, l, r, index):
        if l == r == index:
            self.tree[tree_index] = self.data[index]
            return
        mid = (l+r)//2
        left, right = 2 * tree_index + 1, 2 * tree_index + 2
        if index > mid:
            # 要更新的区间在右子树
            self._update(right, mid+1, r, index)
        else:
            # 要更新的区间在左子树index<=mid
            self._update(left, l, mid, index)
        # 里面的小区间变化了，包裹的大区间也要更新
        self.tree[tree_index] = self._merge(self.tree[left], self.tree[right])
        

class NumArray:

    def __init__(self, nums: List[int]):
        self.segment_tree = SegmentTree(nums, lambda x, y : x + y)

    def update(self, i: int, val: int) -> None:
        self.segment_tree.update(i, val)
        
    def sumRange(self, i: int, j: int) -> int:
        return self.segment_tree.query(i, j)   